In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns
import pandas as pd
sns.set_style("whitegrid")
import matplotlib.colors as colors
GOLDENRATIO = (1 + 5 ** 0.5) / 2
GOLDEN = (9, 9/GOLDENRATIO)
import skrf as rf
#rf.stylely() #nicer styling
%matplotlib inline


In [1]:
LNC_6_20C = #read your network here

LNA_S = LNC_6_20C.s #extracting the Sii values of the network
imaginary_LNA_S = LNA_S.imag #extracting only the imaginary part of each Sii
imaginary_S11 = imaginary_LNA_S[:,1,1] #taking only the S11 at the first port zii

def compute_imaginary_criteria(imaginary_LNA_S11):
    imaginary_criteria = np.sum(np.abs(imaginary_LNA_S11)) #this is the parameter to minimize during the optimization. It's minimum is a point in the real axis of the smith chart = line de-embedded
    return imaginary_criteria

#initialization values
fixed_permittivity_guess = 2.1
initial_length_guess = 1400#7.5#mm
delta_length = 1#0.001#0.0015 #mm
maximum_length_possible = 1600#8.5#mm. ~physical length of the lna housing
list_possible_lengths = np.arange(initial_length_guess,maximum_length_possible,delta_length) #in the optimization, I iterate through these values

current_length = initial_length_guess
freq = LNC_6_20C.frequency
unknown_material = rf.Freespace(frequency=freq, ep_r=fixed_permittivity_guess,z0=50)

imaginary_LNA_S11_list = [] #list to be filled with the imaginary impedances calculated, the optimal value is the index of the minimum in this list

for length in list_possible_lengths:
    #Compute the new impedance after removing this length:
    deembedded_LNC_6_20C = LNC_6_20C**unknown_material.line(d=length,unit='mm').inv#unknown_material.line(d=length,unit='mm').inv**LNC_6_20C#
    LNA_S = deembedded_LNC_6_20C.s #extracting the S11 values of the network
    imaginary_LNA_S = LNA_S.imag #extracting only the imaginary part of each impedance
    imaginary_LNA_S11 = imaginary_LNA_S[:,1,1] #taking only the impedance at the first port zii
    #recalculate the target optimization parameter
    imaginary_criteria_value = compute_imaginary_criteria(imaginary_LNA_S11)

    imaginary_LNA_S11_list.append(imaginary_criteria_value)

minimum_LNA_S11_index = np.argmin(np.asarray(imaginary_LNA_S11_list)) #extracting the argument of the minimum imaginary Sii
minimum_LNA_S11_index
optimal_length = list_possible_lengths[minimum_LNA_S11_index]

print("Optimal Length: {} mm".format(optimal_length))
print("Minimum Imaginary Criteria Value/# of points: {}".format(np.min(np.asarray(imaginary_LNA_S11_list))/len(freq)))

NameError: name 'LNA_networks' is not defined

In [ ]:
plt.figure(figsize=GOLDEN)
plt.title('De-embedding internal length of LNA from S11 measurement',fontweight='bold')
plt.plot(list_possible_lengths,np.asarray(imaginary_LNA_S11_list)/len(freq),label='Possible lengths')
plt.scatter(optimal_length,np.min(np.asarray(imaginary_LNA_S11_list))/len(freq),color='red',label='Optimal value')
plt.ylabel('Im(S11) per datapoint on average')
plt.xlabel('Possible internal length sweep (mm)')
plt.legend()
plt.savefig('C:/Users/MADMAX/Desktop/Juan/thesis_images/cold_LNA_deembed.png')
plt.show()

In [ ]:
deembedded_LNC_6_20C = LNC_6_20C**unknown_material.line(d=optimal_length,unit='mm').inv

fig = plt.figure(figsize=(12,6))
fig.suptitle('de-embedding at 4K of LNA internal length with {} mm and e_r = {}'.format(np.round(optimal_length,3),fixed_permittivity_guess),fontweight='bold',fontsize=16)
plt.subplot(1,2,1)
LNC_6_20C.plot_s_smith(m=1,n=1,label='S11 before')
deembedded_LNC_6_20C.plot_s_smith(m=1,n=1,label='S11 after')

plt.subplot(1,2,2)
LNC_6_20C.plot_z_mag(m=1,n=1,label='Z11 before')
deembedded_LNC_6_20C.plot_z_mag(m=1,n=1,label='Z11 after')
mean_impedance = np.round(np.mean(deembedded_LNC_6_20C.z_mag[:,1,1]),0)
plt.axhline(mean_impedance,label='Mean impedance = {} Ohm'.format(int(mean_impedance)),color='orange',linestyle='--')

plt.legend()
#plt.savefig('C:/Users/MADMAX/Desktop/Juan/thesis_images/cold_LNA_deembed_3_ColdResult.png')
plt.show()